In [9]:
import pandas as pd
import numpy as np
import re

from torch import nn
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import *
import torchvision.models as models


import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from torch.nn import Transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# import pre-processed data
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import keras_toolkit as kt
import keras.backend as K
from sklearn.model_selection import train_test_split
from textwrap import wrap


In [10]:
from text_process import process_text
training_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/train.csv')
val_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/val.csv')
testing_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/15_after_processed/test.csv')
training_dataset, val_dataset, testing_dataset = process_text(training_dataset, val_dataset, testing_dataset)
training_dataset.head()

,posting_id,image,image_phash,title,label_group,proc_title
0,train_2249428809,40d304f91be807fed3b796a158a034f7.jpg,bc69d33e86b491c1,AMTECH Klem C Set 3 Pcs - Catok Clamp 1 2 3 Inch,3685949317,amtech klem c set pcs catok clamp inch
1,train_1453612941,e05aa71652e2546dbc2a6a2d18cc4aa7.jpg,bcfcc1a80ee930da,Cetakan Kue Pukis 10 Lubang Pancong Teflon Ran...,2236232282,cetakan kue pukis lubang pancong teflon rangin...
2,train_414717665,0c0f97f726132be83c7b84e0ca9e604e.jpg,bc4aa5a5de34d2e0,(COD)TAS TOTE BAG T76 TAS CANVAS FASHION WANIT...,3145779110,codtas tote bag t tas canvas fashion wanita im...
3,train_404098311,fedf1394058d6e439698356c0be6f4d6.jpg,cbceb1a14e8bf046,(1kg=7pcs) MEIRA CARDIGAN CARDI OUTHER TANPA J...,4171236554,kgpcs meira cardigan cardi outher tanpa jilbab
4,train_653040440,cfb08184ebbef7a5409e1bd1a0f6d0d9.jpg,c44eec61e516d2b9,Wardah Lightening Day Cream 30gr or night cre...,1774190279,wardah lightening day cream gr night cream gr


In [11]:
dist_model = SentenceTransformer('stsb-distilbert-base')
dist_model.max_seq_length = 128

train_titles = training_dataset.proc_title.tolist()
train_embed = dist_model.encode(train_titles, show_progress_bar=True, convert_to_tensor=True)

val_titles = val_dataset.proc_title.tolist()
val_embed = dist_model.encode(val_titles, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 172/172 [00:01<00:00, 115.96it/s]


In [12]:
IMG_SIZE = 224
size = (IMG_SIZE,IMG_SIZE)
img_model = tf.keras.applications.MobileNet(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = 'imagenet' )

In [13]:
def get_textEmbeddings(model,text):
    text_embedding = model.encode(text, convert_to_tensor=True)
    return text_embedding

def get_imageEmbeddings(model,imagePath):
    image = tf.keras.preprocessing.image.load_img(imagePath,target_size= size)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    img_embeddings = model(input_arr)
    meanImgEmb1 = np.mean(img_embeddings,axis =0)
    meanImgEmb2 = np.mean(meanImgEmb1,axis=0)
    meanImgEmb = np.mean(meanImgEmb2,axis=0)
    return meanImgEmb

X_train, X_val, y_train, y_val = train_embed, val_embed, training_dataset.label_group, val_dataset.label_group

In [21]:
text_embeddings={}
image_embeddings={}
for index,row in  training_dataset.iterrows():
    txt_emb = get_textEmbeddings(dist_model,str(row['proc_title']))
    imagePath = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'+row[1]
    img_emb = get_imageEmbeddings(img_model,imagePath)
    text_embeddings[row[0]] = txt_emb
    image_embeddings[row[0]] = img_emb


In [ ]:
keyList=[]
cembList=[]
imageList=[]
titleList=[]
for index, row in training_dataset.iterrows():
    txt_emb = text_embeddings[row[0]]
    print(np.shape(txt_emb))
    img_emb = image_embeddings[row[0]]
    print(np.shape(img_emb))
    cmb_emb = np.concatenate((txt_emb,img_emb),axis=0)
    print(np.shape(cmb_emb))
    norm = np.linalg.norm(cmb_emb)
    cmb_emb_normal = cmb_emb/norm
    keyList.append(row[0])
    cembList.append(cmb_emb_normal)
    imageList.append(row['image'])
    titleList.append(row['proc_title'])

In [19]:
training_dataset

,posting_id,image,image_phash,title,label_group,proc_title
0,train_2249428809,40d304f91be807fed3b796a158a034f7.jpg,bc69d33e86b491c1,AMTECH Klem C Set 3 Pcs - Catok Clamp 1 2 3 Inch,3685949317,amtech klem c set pcs catok clamp inch
1,train_1453612941,e05aa71652e2546dbc2a6a2d18cc4aa7.jpg,bcfcc1a80ee930da,Cetakan Kue Pukis 10 Lubang Pancong Teflon Ran...,2236232282,cetakan kue pukis lubang pancong teflon rangin...
2,train_414717665,0c0f97f726132be83c7b84e0ca9e604e.jpg,bc4aa5a5de34d2e0,(COD)TAS TOTE BAG T76 TAS CANVAS FASHION WANIT...,3145779110,codtas tote bag t tas canvas fashion wanita im...
3,train_404098311,fedf1394058d6e439698356c0be6f4d6.jpg,cbceb1a14e8bf046,(1kg=7pcs) MEIRA CARDIGAN CARDI OUTHER TANPA J...,4171236554,kgpcs meira cardigan cardi outher tanpa jilbab
4,train_653040440,cfb08184ebbef7a5409e1bd1a0f6d0d9.jpg,c44eec61e516d2b9,Wardah Lightening Day Cream 30gr or night cre...,1774190279,wardah lightening day cream gr night cream gr
...,...,...,...,...,...,...
21915,train_24545330,b8edb73fcf87b7bb3d38066055ad5313.jpg,daa5254a255b5a1f,[Bayar Di Tempat]O.TWO.O Foundation / Conceale...,2804650647,[bayar di tempat]otwoo foundation concealer fo...
21916,train_2854292546,b1c978e1c0e3268629ee15f233664a82.jpg,c5d44f0f27bb5308,Bibit Pemutih Cepuk (Grade B),128996462,bibit pemutih cepuk grade b
21917,train_2330340220,eba8fea4bbf9c4d78015d3b4fe1076c3.jpg,e996966996396368,CANTOLAN TEMPEL gantungan hook perekat tanpa p...,2609303586,cantolan tempel gantungan hook perekat tanpa p...
21918,train_1643562750,3226f7ad71b8049b69e5123d40e7e8be.jpg,b333cccccccccc89,(COD) Skin Aqua SPF 50+ PA+++ Original 100%,2123332638,cod skin aqua spf pa original
